In [1]:
import sagemaker
from sagemaker import get_execution_role

import numpy as np
import pandas as pd

import boto3
import re

print('Libraries_loaded')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Libraries_loaded


In [2]:
bucket_name = 'cloud-project-bucket' #  bucket name 

train_data = r'diabetespred/training/'
val_data = r'diabetespred/validation/' 

s3_model_output_location = r's3://{0}/diabetespred/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,train_data)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,val_data)

In [3]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)

s3://cloud-project-bucket/diabetespred/model
s3://cloud-project-bucket/diabetespred/training/
s3://cloud-project-bucket/diabetespred/validation/


In [4]:
data = 's3://cloud-project-bucket/diabetes.csv'

In [5]:
df = pd.read_csv(data)
df.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


(768, 9)

In [6]:
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols

['Outcome',
 'Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [9]:
df = df[cols]
df.head(5)

,Outcome,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,6,148,72,35,0,33.6,0.627,50
1,0,1,85,66,29,0,26.6,0.351,31
2,1,8,183,64,0,0,23.3,0.672,32
3,0,1,89,66,23,94,28.1,0.167,21
4,1,0,137,40,35,168,43.1,2.288,33


In [10]:
train = df[:629]
val = df[629:]

In [11]:
train.shape

(629, 9)

In [12]:
val.shape

(139, 9)

In [13]:
train.head()

,Outcome,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,6,148,72,35,0,33.6,0.627,50
1,0,1,85,66,29,0,26.6,0.351,31
2,1,8,183,64,0,0,23.3,0.672,32
3,0,1,89,66,23,94,28.1,0.167,21
4,1,0,137,40,35,168,43.1,2.288,33


In [14]:
train.isnull().sum()

Outcome                     0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [15]:
np_train = train.values
np_val = val.values

In [16]:
type(np_train)

numpy.ndarray

In [17]:
np.savetxt("train.csv", np_train, delimiter=",")

In [18]:

np.savetxt("val.csv", np_val, delimiter=",")

In [19]:
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: 
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [20]:
write_to_s3('train.csv', 
            bucket_name,
            train_data + 'train.csv')

write_to_s3('val.csv',
            bucket_name,
            val_data + 'val.csv')

In [21]:
sess = sagemaker.Session()

In [22]:
role = get_execution_role()
role

'arn:aws:iam::619071353841:role/service-role/AmazonSageMaker-ExecutionRole-20241207T213472'

In [23]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(
    sess.boto_region_name,
    "xgboost", 
    "latest")

print(' SageMaker XGBoost Info :\n{} ({})'.format(container, sess.boto_region_name))

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


 SageMaker XGBoost Info :
811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest (us-east-1)


In [30]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.m4.xlarge',
    output_path=s3_model_output_location,
    sagemaker_session=sess,
    base_job_name ='v1-xgboost-diabetes')

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [31]:
estimator.set_hyperparameters(max_depth=7,
                              objective="binary:logistic",num_round=1000)

In [32]:
estimator.hyperparameters()

{'max_depth': 7, 'objective': 'binary:logistic', 'num_round': 1000}

In [33]:
training_input_config = sagemaker.session.s3_input(
    s3_data=s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.s3_input(
    s3_data=s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [34]:
print(training_input_config.config)
print(validation_input_config.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://cloud-project-bucket/diabetespred/training/', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://cloud-project-bucket/diabetespred/validation/', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}


In [35]:
estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: v1-xgboost-diabetes-2024-12-08-04-07-56-863


2024-12-08 04:07:58 Starting - Starting the training job...
2024-12-08 04:08:13 Starting - Preparing the instances for training...
2024-12-08 04:08:44 Downloading - Downloading input data...
2024-12-08 04:09:19 Downloading - Downloading the training image......
2024-12-08 04:10:25 Training - Training image download completed. Training in progress.
2024-12-08 04:10:25 Uploading - Uploading generated training modelArguments: train
[2024-12-08:04:10:21:INFO] Running standalone xgboost training.
[2024-12-08:04:10:21:INFO] File size need to be processed in the node: 0.16mb. Available memory size in the node: 8451.13mb
[2024-12-08:04:10:21:INFO] Determined delimiter of CSV input is ','
[04:10:21] S3DistributionType set as FullyReplicated
[04:10:21] 629x8 matrix with 5032 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-12-08:04:10:21:INFO] Determined delimiter of CSV input is ','
[04:10:21] S3DistributionType set as FullyReplicated
[04:10:21] 139x8 mat

[770]#011train-error:0#011validation-error:0.251799
[04:10:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[771]#011train-error:0#011validation-error:0.251799
[04:10:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[772]#011train-error:0#011validation-error:0.251799
[04:10:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[773]#011train-error:0#011validation-error:0.251799
[04:10:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[774]#011train-error:0#011validation-error:0.251799
[04:10:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[775]#011train-error:0#011validation-error:0.251799
[04:10:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[776]#011train-error:0


2024-12-08 04:10:44 Completed - Training job completed
Training seconds: 120
Billable seconds: 120


In [38]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.t2.medium',
                             endpoint_name = 'v2-xgboost-diabetes')

INFO:sagemaker:Creating model with name: v1-xgboost-diabetes-2024-12-08-04-22-28-592
INFO:sagemaker:Creating endpoint-config with name v2-xgboost-diabetes
INFO:sagemaker:Creating endpoint with name v2-xgboost-diabetes


--------!

In [39]:
data = [4,9.4,0.5,2,0,2,1,2]

In [47]:
# from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()
# predictor.serializer = csv_serializer
# predictor.deserializer = None

In [48]:
result = predictor.predict(data)

In [49]:
np.round(float(result))

0.0